### Twitter API Access

Twitter implements OAuth 1.0A as its standard authentication mechanism and in order to use it to make requests to Twitter's API you'll need to go to https://dev.twitter.com/apps and create a sample application. 

Choose any name for your application, write a description and use http://google.com for the website. 

Under **Key and Access Tokens** there are four primary identifiers to note for an OAuth 1.0A workflow: 
-consumer key
-consumer secret
-access token, and 
-access token secret (Click on Create Access Token to create those)

Note that you will need an ordinary Twitter account in order to login, create an app, and get these credentials.

The first time you execute the notebook, add all credentials so that you can save them in the pkl file, then you can remove the secret keys from the notebook because they will just be loaded from the pkl file.

The pkl file contains sensitive information that can be used to take control of your twitter acccount, do not share it.

In [6]:
import pickle
import os

In [14]:
if not os.path.exists('secret_twitter_credentials.pkl'):
    Twitter = {}
    Twitter['Consumer Key'] = ''
    Twitter['Consumer Secret'] = ''
    Twitter['Access Token'] = ''
    Twitter['Access Token Secret'] = ''
    with open('secret_twitter_credentials.pkl', 'wb') as f:
        pickle.dump(Twitter, f)
else:
    Twitter = pickle.load(open('secret_twitter_credentials.pkl', 'rb'))
    
#here created a Twitter object and stored all of our access credentials, whichh
#allowed us to then delete the actual key and secret information
#from first part of code block after running it the first time
#this Twitter object now stores the credentials for any time we use in the future


#pickle is a cute name for a python utility module to save any Python object
#or data structure on disk. It does something special called serialization. 
#To convert any python object (or in this case, twitter object) into 
#a character stream so the object can be created later in Python when needed
#Secondary benefit of this is to minimize file size 

#Reconstruction of the object is called de-serialization

In [15]:
#install twitter package to interface with the twitter api

!pip install twitter

### Example 1. Authorizing an application to access Twitter account data

In [21]:
import twitter

#create ew object called 'auth' and will use this to create a twitter api object
auth = twitter.oauth.OAuth(Twitter['Access Token'],
                           Twitter['Access Token Secret'],
                           Twitter['Consumer Key'],
                           Twitter['Consumer Secret'])

twitter_api = twitter.Twitter(auth = auth)

#Nothing to see by displaying twitter_api except that it's not a defined variable

print(twitter_api)

#So we've imported twitter package, called twitter, used auth function to 
#create authentication object
#Now use the twitter api object to start accessing the data via the twitter api

### Example 2. Retrieving Trends

By Trends we mean popular hashtags

Twitter identifies locations using the Yahoo! Where on Earth ID. 

This ID for the entire world is 1. See https://dev.twitter.com/docs/api/1.1/get/trends/place and http://developer.yahoo.com/geo/geoplanet/

Here well retrieve a location's specific trends and display the response using a package for JSON data format. JSON is a semi-structured data format that is used to pass information in internet application. It's roughly equivalent to nested python dictionaries and lists. 

Then, we'll use the data to create Python sets that we can run intersection operations over. 

In [17]:
WORLD_WOE_ID = 1
US_WOE_ID = 23424977

Look for the WOEID for san diego: http://woeid.rosselliot.co.nz/lookup/san%20diego%20%20ca

You can change it to another location

In [30]:
LOCAL_WOE_ID = 2487889

#Prefix ID with the underscore for query string parameterization
#Without the underscore, the twitter package appends the ID value
#to the URL itself as a special case keyword argument

world_trends = twitter_api.trends.place(_id=WORLD_WOE_ID)
us_trends = twitter_api.trends.place(_id=US_WOE_ID)
local_trends = twitter_api.trends.place(_id=LOCAL_WOE_ID)

#Using trends.place for twitter api object, we can get the top 50 trends for any location.

#What we received back is a trend object with a response that's in JSON. 

In [29]:
world_trends[:2]

[{'as_of': '2018-04-16T20:44:45Z',
  'created_at': '2018-04-16T20:39:28Z',
  'locations': [{'name': 'Worldwide', 'woeid': 1}],
  'trends': [{'name': 'Sean Hannity',
    'promoted_content': None,
    'query': '%22Sean+Hannity%22',
    'tweet_volume': 236636,
    'url': 'http://twitter.com/search?q=%22Sean+Hannity%22'},
   {'name': 'Pulitzer',
    'promoted_content': None,
    'query': 'Pulitzer',
    'tweet_volume': 110618,
    'url': 'http://twitter.com/search?q=Pulitzer'},
   {'name': '#غبار_الرياض',
    'promoted_content': None,
    'query': '%23%D8%BA%D8%A8%D8%A7%D8%B1_%D8%A7%D9%84%D8%B1%D9%8A%D8%A7%D8%B6',
    'tweet_volume': 32969,
    'url': 'http://twitter.com/search?q=%23%D8%BA%D8%A8%D8%A7%D8%B1_%D8%A7%D9%84%D8%B1%D9%8A%D8%A7%D8%B6'},
   {'name': '#isola',
    'promoted_content': None,
    'query': '%23isola',
    'tweet_volume': 14718,
    'url': 'http://twitter.com/search?q=%23isola'},
   {'name': 'True Thompson',
    'promoted_content': None,
    'query': '%22True+Thompson%2

In [25]:
trends = local_trends
print(type(trends))
print(list(trends[0].keys()))
print(trends[0]['trends'])

<class 'twitter.api.TwitterListResponse'>
['trends', 'as_of', 'created_at', 'locations']
[{'name': 'Sean Hannity', 'url': 'http://twitter.com/search?q=%22Sean+Hannity%22', 'promoted_content': None, 'query': '%22Sean+Hannity%22', 'tweet_volume': 236636}, {'name': '#HeroConf', 'url': 'http://twitter.com/search?q=%23HeroConf', 'promoted_content': None, 'query': '%23HeroConf', 'tweet_volume': None}, {'name': '#SDinMX', 'url': 'http://twitter.com/search?q=%23SDinMX', 'promoted_content': None, 'query': '%23SDinMX', 'tweet_volume': None}, {'name': '#BostonMarathon', 'url': 'http://twitter.com/search?q=%23BostonMarathon', 'promoted_content': None, 'query': '%23BostonMarathon', 'tweet_volume': 74624}, {'name': '#MondayMotivation', 'url': 'http://twitter.com/search?q=%23MondayMotivation', 'promoted_content': None, 'query': '%23MondayMotivation', 'tweet_volume': 213173}, {'name': '#Beychella', 'url': 'http://twitter.com/search?q=%23Beychella', 'promoted_content': None, 'query': '%23Beychella', 't

### Example 3. Displaying API responses as pretty-printed JSON

In [40]:
import json

#Here we're saying indent every new parenthesis, or every new level as its called in JSON with one character
#Dumps creates a string from the dictionary (indicated by 's' at the end of dump)
#Dump is used to dump a dictionary into a file

print((json.dumps(us_trends[:2], indent=1)))

[
 {
  "trends": [
   {
    "name": "Sean Hannity",
    "url": "http://twitter.com/search?q=%22Sean+Hannity%22",
    "promoted_content": null,
    "query": "%22Sean+Hannity%22",
    "tweet_volume": 309755
   },
   {
    "name": "Pulitzer",
    "url": "http://twitter.com/search?q=Pulitzer",
    "promoted_content": null,
    "query": "Pulitzer",
    "tweet_volume": 146925
   },
   {
    "name": "True Thompson",
    "url": "http://twitter.com/search?q=%22True+Thompson%22",
    "promoted_content": null,
    "query": "%22True+Thompson%22",
    "tweet_volume": 38457
   },
   {
    "name": "#NationalVolunteerWeek",
    "url": "http://twitter.com/search?q=%23NationalVolunteerWeek",
    "promoted_content": null,
    "query": "%23NationalVolunteerWeek",
    "tweet_volume": 10907
   },
   {
    "name": "#DontConfuseMeFor",
    "url": "http://twitter.com/search?q=%23DontConfuseMeFor",
    "promoted_content": null,
    "query": "%23DontConfuseMeFor",
    "tweet_volume": null
   },
   {
    "name": 

In [28]:
print((json.dumps(trends[0], indent =1)))

{
 "trends": [
  {
   "name": "Sean Hannity",
   "url": "http://twitter.com/search?q=%22Sean+Hannity%22",
   "promoted_content": null,
   "query": "%22Sean+Hannity%22",
   "tweet_volume": 236636
  },
  {
   "name": "#HeroConf",
   "url": "http://twitter.com/search?q=%23HeroConf",
   "promoted_content": null,
   "query": "%23HeroConf",
   "tweet_volume": null
  },
  {
   "name": "#SDinMX",
   "url": "http://twitter.com/search?q=%23SDinMX",
   "promoted_content": null,
   "query": "%23SDinMX",
   "tweet_volume": null
  },
  {
   "name": "#BostonMarathon",
   "url": "http://twitter.com/search?q=%23BostonMarathon",
   "promoted_content": null,
   "query": "%23BostonMarathon",
   "tweet_volume": 74624
  },
  {
   "name": "#MondayMotivation",
   "url": "http://twitter.com/search?q=%23MondayMotivation",
   "promoted_content": null,
   "query": "%23MondayMotivation",
   "tweet_volume": 213173
  },
  {
   "name": "#Beychella",
   "url": "http://twitter.com/search?q=%23Beychella",
   "promoted_c

### Example 4. Computing the intersection of two sets of trends

In [44]:
#Getting the name ofr all trends using a loop: using world label, san diego, and us

trends_set = {}
trends_set['world'] = set([trend['name']
                          for trend in world_trends[0]['trends']])

trends_set['us'] = set([trend['name']
                       for trend in us_trends[0]['trends']])

trends_set['san diego'] = set([trend['name']
                              for trend in local_trends[0]['trends']])

#now have a trends_set object (dictionary) with three sets in it
                                                 

In [46]:
#Now first creating a for loop that joins all trends for a particular location and prints them in pretty format

for loc in ['world', 'us', 'san diego']:
    print(('-'*10, loc))
    print((','.join(trends_set[loc])))

('----------', 'world')
Carlos Slim,Sean Hannity,#FamaABailar16A,#SözününEri,#MadeInChelsea,#بلغنا_رمضان_يا_الله,#SanırımBen,#jajnec,El Juli,#الهلال_الاستقلال,#lanita8tv,#LoveYourselfTearIsComing,#BonoDeIndependencia,#MTVLACARACYP,#TheQueensGreenPlanet,#M05SCF,#OjosQueNoVen,West Ham,#غبار_الرياض,#UniversityChallenge,#Clem,#TheIsland,#rtlln,#المملكه_درع_الخليج,#Scorpion,Pulitzer,#جامعه_القصيم,#KadınlarınYaptığıHatalar,#TheWalkingDeadUK,#babalarveoğulları,#4DaysUntilArianaReturns,Joe Hart,#SouLuna3,True Thompson,#elintermedio,#HarryStylesLiveOnTourDublin,#بطفولتك_كنت_هادي_والا_شقي,#BostonMarathon,#Bizimİçin,#LaVozKidsBatallas3,#NationalVolunteerWeek,#conquis14,#LateNightBerlin,#RenunciaMagario,#DontConfuseMeFor,#SB29FCL,#HappyFirstDates,#ottoemezzo,#KKRvDD,#isola
('----------', 'us')
#WHUSTK,Sean Hannity,#DearWhitePeople,#AzureSphere,Desiree Linden,Vic Bubas,Vince Staples,#WhenAnimalsParty,#DeadlineWH,#PRETTYMUCHEP,PEGOT,#Scorpion,Hamidou Diallo,Stoke,Pulitzer,Harvey Weinstein,J Cole,Jud

In [37]:
print(('='*10, 'intersection of world and us'))
print((trends_set['world'].intersection(trends_set['us'])))

print(('='*10, 'intersection of us and san-diego'))
print((trends_set['san diego'].intersection(trends_set['us'])))

('==========', 'intersection of world and us')
{'True Thompson', 'Sean Hannity', '#4DaysUntilArianaReturns', '#DontConfuseMeFor', '#NationalVolunteerWeek', '#Scorpion', 'Joe Hart', 'Pulitzer'}
('==========', 'intersection of us and san-diego')
{'#WHUSTK', 'Sean Hannity', '#DearWhitePeople', '#AzureSphere', 'Vic Bubas', '#WhenAnimalsParty', 'Vince Staples', '#DeadlineWH', '#PRETTYMUCHEP', 'PEGOT', '#Scorpion', 'Hamidou Diallo', 'Stoke', 'Pulitzer', 'Harvey Weinstein', 'J Cole', 'Judge Kimba Wood', 'Seven Days of Heroin', '#Hotelpocalypse', 'Kyle Busch', '#MusicMonday', 'Joe Hart', '#4DaysUntilArianaReturns', '#ASUGSV2018', "Bill O'Reilly", 'True Thompson', '#MotivationMonday', 'Adam Pally', 'Summer Jam', '#NationalVolunteerWeek', '#DavidHoggsNextBoycott', '#MyCareerPath', '#DontConfuseMeFor', '#GuaranteedToFindAtWalmart', 'Twitch Prime', '#EarthWeek', 'Bubba Wallace'}
